In [ ]:
!pip install -q langchain faiss-cpu sentence-transformers==2.2.2 InstructorEmbedding pypdf

In [ ]:
pip install google-cloud-aiplatform

In [1]:
from langchain.document_loaders import TextLoader
from pypdf import PdfReader
from langchain import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory

import functools
import time
from concurrent.futures import ThreadPoolExecutor
from typing import Generator, List, Tuple
from typing import Optional, TypeVar
import math
from typing import Any
from pathlib import Path
import os
import json
import numpy as np

from vertexai.preview.language_models import TextEmbeddingModel

model = TextEmbeddingModel.from_pretrained("textembedding-gecko")

# Define an embedding method that uses the model
def encode_texts_to_embeddings(sentences: List[str]) -> List[Optional[List[float]]]:
    print("batch size ", len(sentences))
    try:
        embeddings = model.get_embeddings(sentences)
        return [embedding.values for embedding in embeddings]
    except Exception as e:
        print("exception", e)
        return [None for _ in range(len(sentences))]
    

# Generator function to yield batches of sentences
def generate_batches(
    sentences: List[str], batch_size: int
) -> Generator[List[str], None, None]:
    for i in range(0, len(sentences), batch_size):
        yield sentences[i : i + batch_size]


def encode_text_to_embedding_batched(
    sentences: List[str], api_calls_per_second: int = 1, batch_size: int = 100
) -> Tuple[List[bool], np.ndarray]:

    embeddings_list: List[List[float]] = []
    print(len(sentences))
    print(sentences[0])
    # Prepare the batches using a generator
    batches = generate_batches(sentences, batch_size)

    seconds_per_job = 1 / api_calls_per_second

    with ThreadPoolExecutor() as executor:
        futures = []
        for batch in tqdm(
            batches, total=math.ceil(len(sentences) / batch_size), position=0
        ):
            futures.append(
                executor.submit(functools.partial(encode_texts_to_embeddings), batch)
            )
            time.sleep(seconds_per_job)

        for future in futures:
            embeddings_list.extend(future.result())

    is_successful = [
        embedding is not None for sentence, embedding in zip(sentences, embeddings_list)
    ]
    print(is_successful)
    embeddings_list_successful = np.squeeze(
        np.stack([embedding for embedding in embeddings_list if embedding is not None])
    )
    return is_successful, embeddings_list_successful

In [19]:
# block to process entire directory of pdf docs

embeddings_file_path = Path("/home/jupyter/rbi-bot/embeddings/")

# Create a rate limit of 300 requests per minute. Adjust this depending on your quota.
API_CALLS_PER_SECOND = 1
# According to the docs, each request can process 5 instances per request
ITEMS_PER_REQUEST = 100


documents_directory = '/home/jupyter/rbi-bot/rbi-docs'

splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50
)

def split_documents(splitter, documents_1):
    # Split the documents using the provided splitter
    split_1 = splitter.split_text(documents_1)
    # Create documents from the split chunks
    #split_1 = splitter.create_documents(split_1)
    return split_1

vector_db = None
for i, filename in enumerate(os.listdir(documents_directory)):
    documents_1 = ''
    # Create the full file path
    filepath = os.path.join(documents_directory, filename)
    
    chunk_path = embeddings_file_path.joinpath(
        f"{embeddings_file_path.stem}_{filename}.json"
    )
    
    # Check if the file is a file and not a directory
    if os.path.isfile(filepath):
        # Open the file
        with open(filepath, 'r') as file:
            # Read the contents of the file
            print(f"Processing file {i,filename}:")
            reader = PdfReader(filepath)
            for page in reader.pages:
                documents_1 += page.extract_text()
        # Implement embeddings
        print(f"extracted pages")
        splits_1 = split_documents(splitter,documents_1)
        print(f"split documents")
        is_successful, question_chunk_embeddings = encode_text_to_embedding_batched(
        sentences=splits_1,
        api_calls_per_second=API_CALLS_PER_SECOND,
        batch_size=ITEMS_PER_REQUEST,
        )
        print(f"embeddings generated :", is_successful)
        # Save embeddings to jsonl format
        print(f"saving to file :")
        with open(chunk_path, "a") as f:
            # Append to file
            embeddings_formatted = [
                json.dumps(
                    {
                        "id": filename + "_" + str(num),
                        "embedding": [str(value) for value in embedding],
                    }
                )
                + "\n"
                for num,embedding in enumerate(question_chunk_embeddings)
            ]
            f.writelines(embeddings_formatted)
        print(f"saved")

